---
title: "LLM Evals: 5 lessons of what actually works"
author: "Safouane Chergui"
date: "2025-11-19"
format: html
toc: true
toc-location: body
toc-depth: 4
categories: [LLM, Evaluation]
---

## Introduction

Deploying LLMs in production is the easy part. The hard part? Making sure they're actually working.

I've been deploying LLMs in production in many companies and across different industries for almost 3 years now. Each time, I encountered the same critical challenge: **how do you truly evaluate whether your LLM is performing well and not only rely on vibe checks ?**

If you're struggling to move beyond "it looks good to me" when evaluating your LLM applications, this blog post is for you.

<br><br>
<div align="center">
<img src="./assets/front_image.jpg" width="80%" style="display: block; margin: 0 auto;">
</div>
<br>

This blog post contains lessons learnt through hands-on experience. These lessons come mostly from my experience testing what I have learnt in deploying real-life LLM applications, talking with peers, doing courses, and reading blog posts (I'll link to these resources as we go along).

Each lesson will tackle a specific part of building and evaluating real LLM applications.

# Lesson 1: I'll vibe-check my app

A classic usecase that I've seen is a customer that wants to build a RAG over products documentation. Once a vanilla RAG is built over the internal documentation, the customer wants to evaluate it and to iterate over it.


People expects anything related to AI these days to go fast and to "just work".
This can be seen in the way most people do evals.

While working with so many customers, I've seen them promote this idea of doing "vibe checks" to test the app where you ask a few questions to the app and see if the answers look good to you.

This is a terrible way to evaluate your LLM applications for multiple reasons:



# Lesson 2: I don't have any data to test my application, where do I start ?

**Sub-lesson 1: Real data is better than synthetic data**

**Sub-lesson 2: The bad way to create synthetic data**

**Sub-lesson 3: The good way to create synthetic data**


# Lesson 3: Have a systematic way to identify failure modes

When you start evaluating your LLM applications, you need to have a systematic way to identify failure modes.

# Lesson 4: Off-the-shelf evals don't work

# Lesson 5: The evaluation interface matters